In [ ]:
!pip install matplotlib-venn
!pip install isodate
import requests
import pandas as pd
import isodate


API_KEY = "API key"

SEARCH_URL = "https://www.googleapis.com/youtube/v3/search"
VIDEO_DETAILS_URL = "https://www.googleapis.com/youtube/v3/videos"
channel_id = "UCiVs2pnGW5mLIc1jS2nxhjg"  # ID de la chaîne cible

video_ids = []
next_page_token = None


while True:
    search_params = {
        'part': 'snippet',
        'channelId': channel_id,
        'maxResults': 50,
        'order': 'date',
        'type': 'video',
        'pageToken': next_page_token,
        'key': API_KEY,
    }
    response = requests.get(SEARCH_URL, params=search_params)
    search_data = response.json()

    if 'items' not in search_data:
        print("Erreur dans la récupération des vidéos :", search_data)
        break

    video_ids.extend([item['id']['videoId'] for item in search_data['items']])

    next_page_token = search_data.get('nextPageToken')
    if not next_page_token:
        break

print(f"Nombre de vidéos récupérées : {len(video_ids)}")

video_details = []

def format_duration(iso_duration):
    """Formater la durée ISO 8601 en un format lisible (MM:SS)."""
    try:
        duration = isodate.parse_duration(iso_duration)
        total_seconds = int(duration.total_seconds())
        minutes, seconds = divmod(total_seconds, 60)
        return f"{minutes:02}:{seconds:02}"
    except:
        return "N/A"

for i in range(0, len(video_ids), 50):
    video_id_chunk = ",".join(video_ids[i:i+50])
    details_params = {
        'part': 'snippet,statistics,contentDetails',
        'id': video_id_chunk,
        'key': API_KEY,
    }
    details_response = requests.get(VIDEO_DETAILS_URL, params=details_params)
    details_data = details_response.json()

    if 'items' not in details_data:
        print("Erreur dans la récupération des détails :", details_data)
        continue

    for video in details_data['items']:
        snippet = video['snippet']
        statistics = video.get('statistics', {})
        content_details = video.get('contentDetails', {})

        video_details.append({
            'Video ID': video['id'],
            'Title': snippet['title'],
            'Description': snippet['description'],
            'Published Date': snippet['publishedAt'],
            'Tags': ", ".join(snippet.get('tags', [])),
            'Views': statistics.get('viewCount', 'N/A'),
            'Likes': statistics.get('likeCount', 'N/A'),
            'Comments': statistics.get('commentCount', 'N/A'),
            'Duration': format_duration(content_details.get('duration', 'N/A')),
            'Product Placement': content_details.get('hasCustomThumbnail', False)
        })

print(f"Nombre de vidéos avec détails récupérées : {len(video_details)}")

if video_details:
    df = pd.DataFrame(video_details)
    print("Aperçu des données :", df.head())
    df.to_csv("kimpro_videos_with_placements.csv", index=False)
    print("Les données ont été exportées dans 'kimpro_videos_with_placements.csv'.")
else:
    print("Aucune donnée à exporter.")


Nombre de vidéos récupérées : 502
Nombre de vidéos avec détails récupérées : 502
Aperçu des données :       Video ID          Title  \
0  ueUTuTITCKc  Yes✅️ or No❌️   
1  OyRUao7_vds       어디에 있을까?   
2  5bSXOrUXLn4    아이스크림 나눠 먹기   
3  sPLOC3nHCVw             편애   
4  Kd5nheFLOHg     승자는 누구일까?🤣   

                                         Description        Published Date  \
0  #김프로 #유백합 #KIMPRO\n🐯Contact🍀\nDM : https://www...  2024-12-16T15:20:06Z   
1  #김프로 #유백합 #KIMPRO\n🐯Contact🍀\nDM : https://www...  2024-12-15T15:00:32Z   
2  #김프로 #유백합 #KIMPRO\n🐯Contact🍀\nDM : https://www...  2024-12-15T12:52:33Z   
3  #김프로 #유백합 #KIMPRO\n🐯Contact🍀\nDM : https://www...  2024-12-14T15:33:24Z   
4  #김프로 #유백합 #KIMPRO\n🐯Contact🍀\nDM : https://www...  2024-12-13T15:02:56Z   

                                                Tags    Views   Likes  \
0  김프로, kimpro, 김동준, 김프로유투브, 유투브김프로, 김동준유투브, 유백합,...        0      98   
1  김프로, kimpro, 김동준, 김프로유투브, 유투브김프로, 김동준유투브, 유백합,...  1100549   41868   
2  김프로, kim

In [ ]:
df

,Video ID,Title,Description,Published Date,Tags,Views,Likes,Comments,Duration,Product Placement
0,ueUTuTITCKc,Yes✅️ or No❌️,#김프로 #유백합 #KIMPRO\n🐯Contact🍀\nDM : https://www...,2024-12-16T15:20:06Z,"김프로, kimpro, 김동준, 김프로유투브, 유투브김프로, 김동준유투브, 유백합,...",0,98,10,00:59,False
1,OyRUao7_vds,어디에 있을까?,#김프로 #유백합 #KIMPRO\n🐯Contact🍀\nDM : https://www...,2024-12-15T15:00:32Z,"김프로, kimpro, 김동준, 김프로유투브, 유투브김프로, 김동준유투브, 유백합,...",1100549,41868,94,00:30,False
2,5bSXOrUXLn4,아이스크림 나눠 먹기,#김프로 #유백합 #KIMPRO\n🐯Contact🍀\nDM : https://www...,2024-12-15T12:52:33Z,"김프로, kimpro, 김동준, 김프로유투브, 유투브김프로, 김동준유투브, 유백합,...",1129663,33876,118,00:20,False
3,sPLOC3nHCVw,편애,#김프로 #유백합 #KIMPRO\n🐯Contact🍀\nDM : https://www...,2024-12-14T15:33:24Z,"김프로, kimpro, 김동준, 김프로유투브, 유투브김프로, 김동준유투브, 유백합,...",1988193,155748,362,00:55,False
4,Kd5nheFLOHg,승자는 누구일까?🤣,#김프로 #유백합 #KIMPRO\n🐯Contact🍀\nDM : https://www...,2024-12-13T15:02:56Z,"김프로, kimpro, 김동준, 김프로유투브, 유투브김프로, 김동준유투브, 유백합,...",1992713,74421,142,00:31,False
...,...,...,...,...,...,...,...,...,...,...
497,hHJfoluKQXo,결과를 예상했나요?🤔,#김프로 #유백합 #KIMPRO\n🐯Contact🍀\nDM : https://www...,2023-02-01T06:00:10Z,"김프로, kimpro, 김동준, 김프로틱톡, 김프로유투브, tiktok, 챌린지크리...",507759,19865,100,00:30,False
498,bNgjB2DBP1s,휘파람 도전,#김프로 #유백합 #KIMPRO\n🐯Contact🍀\nDM : https://www...,2023-01-30T12:51:59Z,"김프로, kimpro, 김동준, 김프로틱톡, 김프로유투브, tiktok, 챌린지크리...",158663042,2577992,6907,00:08,False
499,ras3VE2g0xo,빨대의 진실 👀,#김프로 #유백합 #KIMPRO\n🐯Contact🍀\nDM : https://www...,2023-01-29T09:45:02Z,"김프로, kimpro, 김동준, 김프로틱톡, 김프로유투브, tiktok, 챌린지크리...",24052179,586621,1034,00:22,False
500,RuUHyDjKVM4,borzoidog result,#김프로 #유백합 #KIMPRO\n🐯Contact🍀\nDM : https://www...,2023-01-27T14:19:49Z,"김프로, kimpro, 김동준, 김프로틱톡, 김프로유투브, tiktok, 챌린지크리...",9198608,163272,221,00:13,False


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502 entries, 0 to 501
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Video ID           502 non-null    object
 1   Title              502 non-null    object
 2   Description        502 non-null    object
 3   Published Date     502 non-null    object
 4   Tags               502 non-null    object
 5   Views              502 non-null    object
 6   Likes              502 non-null    object
 7   Comments           502 non-null    object
 8   Duration           502 non-null    object
 9   Product Placement  502 non-null    bool  
dtypes: bool(1), object(9)
memory usage: 35.9+ KB


In [ ]:
# modify dtypes "Published Date", "Views", "Likes"
df[["Views", "Likes"]]=df[["Views", "Likes"]].astype(int)
df["Published Date"] = pd.to_datetime(df["Published Date"])
df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502 entries, 0 to 501
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   Video ID           502 non-null    object             
 1   Title              502 non-null    object             
 2   Description        502 non-null    object             
 3   Published Date     502 non-null    datetime64[ns, UTC]
 4   Tags               502 non-null    object             
 5   Views              502 non-null    int64              
 6   Likes              502 non-null    int64              
 7   Comments           502 non-null    object             
 8   Duration           502 non-null    object             
 9   Product Placement  502 non-null    bool               
dtypes: bool(1), datetime64[ns, UTC](1), int64(2), object(6)
memory usage: 35.9+ KB


,Video ID,Title,Description,Published Date,Tags,Views,Likes,Comments,Duration,Product Placement
0,ueUTuTITCKc,Yes✅️ or No❌️,#김프로 #유백합 #KIMPRO\n🐯Contact🍀\nDM : https://www...,2024-12-16 15:20:06+00:00,"김프로, kimpro, 김동준, 김프로유투브, 유투브김프로, 김동준유투브, 유백합,...",0,98,10,00:59,False
1,OyRUao7_vds,어디에 있을까?,#김프로 #유백합 #KIMPRO\n🐯Contact🍀\nDM : https://www...,2024-12-15 15:00:32+00:00,"김프로, kimpro, 김동준, 김프로유투브, 유투브김프로, 김동준유투브, 유백합,...",1100549,41868,94,00:30,False
2,5bSXOrUXLn4,아이스크림 나눠 먹기,#김프로 #유백합 #KIMPRO\n🐯Contact🍀\nDM : https://www...,2024-12-15 12:52:33+00:00,"김프로, kimpro, 김동준, 김프로유투브, 유투브김프로, 김동준유투브, 유백합,...",1129663,33876,118,00:20,False
3,sPLOC3nHCVw,편애,#김프로 #유백합 #KIMPRO\n🐯Contact🍀\nDM : https://www...,2024-12-14 15:33:24+00:00,"김프로, kimpro, 김동준, 김프로유투브, 유투브김프로, 김동준유투브, 유백합,...",1988193,155748,362,00:55,False
4,Kd5nheFLOHg,승자는 누구일까?🤣,#김프로 #유백합 #KIMPRO\n🐯Contact🍀\nDM : https://www...,2024-12-13 15:02:56+00:00,"김프로, kimpro, 김동준, 김프로유투브, 유투브김프로, 김동준유투브, 유백합,...",1992713,74421,142,00:31,False
...,...,...,...,...,...,...,...,...,...,...
497,hHJfoluKQXo,결과를 예상했나요?🤔,#김프로 #유백합 #KIMPRO\n🐯Contact🍀\nDM : https://www...,2023-02-01 06:00:10+00:00,"김프로, kimpro, 김동준, 김프로틱톡, 김프로유투브, tiktok, 챌린지크리...",507759,19865,100,00:30,False
498,bNgjB2DBP1s,휘파람 도전,#김프로 #유백합 #KIMPRO\n🐯Contact🍀\nDM : https://www...,2023-01-30 12:51:59+00:00,"김프로, kimpro, 김동준, 김프로틱톡, 김프로유투브, tiktok, 챌린지크리...",158663042,2577992,6907,00:08,False
499,ras3VE2g0xo,빨대의 진실 👀,#김프로 #유백합 #KIMPRO\n🐯Contact🍀\nDM : https://www...,2023-01-29 09:45:02+00:00,"김프로, kimpro, 김동준, 김프로틱톡, 김프로유투브, tiktok, 챌린지크리...",24052179,586621,1034,00:22,False
500,RuUHyDjKVM4,borzoidog result,#김프로 #유백합 #KIMPRO\n🐯Contact🍀\nDM : https://www...,2023-01-27 14:19:49+00:00,"김프로, kimpro, 김동준, 김프로틱톡, 김프로유투브, tiktok, 챌린지크리...",9198608,163272,221,00:13,False


In [ ]:
# error downloading database in big query
# try drop "Description" column
df_clean = df.drop("Description", axis= 1)
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502 entries, 0 to 501
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   Video ID           502 non-null    object             
 1   Title              502 non-null    object             
 2   Published Date     502 non-null    datetime64[ns, UTC]
 3   Tags               502 non-null    object             
 4   Views              502 non-null    int64              
 5   Likes              502 non-null    int64              
 6   Comments           502 non-null    object             
 7   Duration           502 non-null    object             
 8   Product Placement  502 non-null    bool               
dtypes: bool(1), datetime64[ns, UTC](1), int64(2), object(5)
memory usage: 32.0+ KB


In [ ]:
df_clean.to_csv("kimpro_videos_with_placements.csv", index=False)
df_clean

,Video ID,Title,Published Date,Tags,Views,Likes,Comments,Duration,Product Placement
0,ueUTuTITCKc,Yes✅️ or No❌️,2024-12-16 15:20:06+00:00,"김프로, kimpro, 김동준, 김프로유투브, 유투브김프로, 김동준유투브, 유백합,...",0,98,10,00:59,False
1,OyRUao7_vds,어디에 있을까?,2024-12-15 15:00:32+00:00,"김프로, kimpro, 김동준, 김프로유투브, 유투브김프로, 김동준유투브, 유백합,...",1100549,41868,94,00:30,False
2,5bSXOrUXLn4,아이스크림 나눠 먹기,2024-12-15 12:52:33+00:00,"김프로, kimpro, 김동준, 김프로유투브, 유투브김프로, 김동준유투브, 유백합,...",1129663,33876,118,00:20,False
3,sPLOC3nHCVw,편애,2024-12-14 15:33:24+00:00,"김프로, kimpro, 김동준, 김프로유투브, 유투브김프로, 김동준유투브, 유백합,...",1988193,155748,362,00:55,False
4,Kd5nheFLOHg,승자는 누구일까?🤣,2024-12-13 15:02:56+00:00,"김프로, kimpro, 김동준, 김프로유투브, 유투브김프로, 김동준유투브, 유백합,...",1992713,74421,142,00:31,False
...,...,...,...,...,...,...,...,...,...
497,hHJfoluKQXo,결과를 예상했나요?🤔,2023-02-01 06:00:10+00:00,"김프로, kimpro, 김동준, 김프로틱톡, 김프로유투브, tiktok, 챌린지크리...",507759,19865,100,00:30,False
498,bNgjB2DBP1s,휘파람 도전,2023-01-30 12:51:59+00:00,"김프로, kimpro, 김동준, 김프로틱톡, 김프로유투브, tiktok, 챌린지크리...",158663042,2577992,6907,00:08,False
499,ras3VE2g0xo,빨대의 진실 👀,2023-01-29 09:45:02+00:00,"김프로, kimpro, 김동준, 김프로틱톡, 김프로유투브, tiktok, 챌린지크리...",24052179,586621,1034,00:22,False
500,RuUHyDjKVM4,borzoidog result,2023-01-27 14:19:49+00:00,"김프로, kimpro, 김동준, 김프로틱톡, 김프로유투브, tiktok, 챌린지크리...",9198608,163272,221,00:13,False
